In [1]:
# Importing packages
import numpy as np
from numba import njit
from pychange.segment import create_summary_stats
from pychange.numba_costs import normal_mean_var_cost

In [2]:
# Creating synthetic data
x = np.concatenate([np.random.normal(0, 1, (10000,)),
                    np.random.normal(20, 2, (10000,)),
                    np.random.normal(-10, 1, (10000,)),
                   np.random.normal(10, 1, (10000,))])

In [84]:
def pelt(x, min_len, penalty, cost_fn):
    """Pruned exact linear time changepoint segmentation"""
    
    # Setting up summary statistics and objects
    n = x.shape[0]
    sum_stats = np.stack((np.append(0, x.cumsum()),
                          np.append(0, (x ** 2).cumsum()),
                          np.append(0, ((x - x.mean()) ** 2).cumsum()),
                          np.arange(0, n + 1)),
                         axis=-1)
    
    # Initializing pelt parameters
    f = np.empty(shape=(n,), dtype=np.float64)
    f[0] = -penalty
    cp = [[]]
    r = np.array([0])
    
    
    # Entering main loop
    for tau_star in np.arange(1, n + 1):
        
        # Calculating minimum segment cost
        _costs = f[tau_star] + penalty + cost_fn(sum_stats[tau_star] - sum_stats[r])
        f[tau_star] = _costs.min()
        tau_l = np.argmin(_costs)
        
        # Setting new changepoints
        cp.append(cp[tau_l] + [tau_l])
        
        # Setting new candidate points
        r = np.int32([j for i, j in enumerate(r) if f[i] + _costs[i] <= (f[tau_star] - penalty)])
        r = np.append(r, tau_star)
        
    return cp[-1]

In [85]:
pelt(x, 10, 100, normal_mean_var_cost)

[0 1]
[2]
[2 3]
[4]
[4 5]
[6]
[6 7]
[8]
[8 9]
[10]
[10 11]
[12]
[12 13]
[14]
[14 15]
[16]
[16 17]
[18]
[18 19]
[20]
[20 21]
[22]
[22 23]
[24]
[24 25]
[26]
[26 27]
[28]
[28 29]
[30]
[30 31]
[32]
[32 33]
[34]
[34 35]
[36]
[36 37]
[38]
[38 39]
[40]
[40 41]
[42]
[42 43]
[44]
[44 45]
[46]
[46 47]
[48]
[48 49]
[50]
[50 51]
[52]
[52 53]
[54]
[54 55]
[56]
[56 57]
[58]
[58 59]
[60]
[60 61]
[62]
[62 63]
[64]
[64 65]
[66]
[66 67]
[68]
[68 69]
[70]
[70 71]
[72]
[72 73]
[74]
[74 75]
[76]
[76 77]
[78]
[78 79]
[80]
[80 81]
[82]
[82 83]
[84]
[84 85]
[86]
[86 87]
[88]
[88 89]
[90]
[90 91]
[92]
[92 93]
[94]
[94 95]
[96]
[96 97]
[98]
[98 99]
[100]
[100 101]
[102]
[102 103]
[104]
[104 105]
[106]
[106 107]
[108]
[108 109]
[110]
[110 111]
[112]
[112 113]
[114]
[114 115]
[116]
[116 117]
[118]
[118 119]
[120]
[120 121]
[122]
[122 123]
[124]
[124 125]
[126]
[126 127]
[128]
[128 129]
[130]
[130 131]
[132]
[132 133]
[134]
[134 135]
[136]
[136 137]
[138]
[138 139]
[140]
[140 141]
[142]
[142 143]
[144]
[144 145]
[

[1086 1087]
[1088]
[1088 1089]
[1090]
[1090 1091]
[1092]
[1092 1093]
[1094]
[1094 1095]
[1096]
[1096 1097]
[1098]
[1098 1099]
[1100]
[1100 1101]
[1102]
[1102 1103]
[1104]
[1104 1105]
[1106]
[1106 1107]
[1108]
[1108 1109]
[1110]
[1110 1111]
[1112]
[1112 1113]
[1114]
[1114 1115]
[1116]
[1116 1117]
[1118]
[1118 1119]
[1120]
[1120 1121]
[1122]
[1122 1123]
[1124]
[1124 1125]
[1126]
[1126 1127]
[1128]
[1128 1129]
[1130]
[1130 1131]
[1132]
[1132 1133]
[1134]
[1134 1135]
[1136]
[1136 1137]
[1138]
[1138 1139]
[1140]
[1140 1141]
[1142]
[1142 1143]
[1144]
[1144 1145]
[1146]
[1146 1147]
[1148]
[1148 1149]
[1150]
[1150 1151]
[1152]
[1152 1153]
[1154]
[1154 1155]
[1156]
[1156 1157]
[1158]
[1158 1159]
[1160]
[1160 1161]
[1162]
[1162 1163]
[1164]
[1164 1165]
[1166]
[1166 1167]
[1168]
[1168 1169]
[1170]
[1170 1171]
[1172]
[1172 1173]
[1174]
[1174 1175]
[1176]
[1176 1177]
[1178]
[1178 1179]
[1180]
[1180 1181]
[1182]
[1182 1183]
[1184]
[1184 1185]
[1186]
[1186 1187]
[1188]
[1188 1189]
[1190]
[1190 1191]


[2198]
[2198 2199]
[2200]
[2200 2201]
[2202]
[2202 2203]
[2204]
[2204 2205]
[2206]
[2206 2207]
[2208]
[2208 2209]
[2210]
[2210 2211]
[2212]
[2212 2213]
[2214]
[2214 2215]
[2216]
[2216 2217]
[2218]
[2218 2219]
[2220]
[2220 2221]
[2222]
[2222 2223]
[2224]
[2224 2225]
[2226]
[2226 2227]
[2228]
[2228 2229]
[2230]
[2230 2231]
[2232]
[2232 2233]
[2234]
[2234 2235]
[2236]
[2236 2237]
[2238]
[2238 2239]
[2240]
[2240 2241]
[2242]
[2242 2243]
[2244]
[2244 2245]
[2246]
[2246 2247]
[2248]
[2248 2249]
[2250]
[2250 2251]
[2252]
[2252 2253]
[2254]
[2254 2255]
[2256]
[2256 2257]
[2258]
[2258 2259]
[2260]
[2260 2261]
[2262]
[2262 2263]
[2264]
[2264 2265]
[2266]
[2266 2267]
[2268]
[2268 2269]
[2270]
[2270 2271]
[2272]
[2272 2273]
[2274]
[2274 2275]
[2276]
[2276 2277]
[2278]
[2278 2279]
[2280]
[2280 2281]
[2282]
[2282 2283]
[2284]
[2284 2285]
[2286]
[2286 2287]
[2288]
[2288 2289]
[2290]
[2290 2291]
[2292]
[2292 2293]
[2294]
[2294 2295]
[2296]
[2296 2297]
[2298]
[2298 2299]
[2300]
[2300 2301]
[2302]
[2302

[3066 3067]
[3068]
[3068 3069]
[3070]
[3070 3071]
[3072]
[3072 3073]
[3074]
[3074 3075]
[3076]
[3076 3077]
[3078]
[3078 3079]
[3080]
[3080 3081]
[3082]
[3082 3083]
[3084]
[3084 3085]
[3086]
[3086 3087]
[3088]
[3088 3089]
[3090]
[3090 3091]
[3092]
[3092 3093]
[3094]
[3094 3095]
[3096]
[3096 3097]
[3098]
[3098 3099]
[3100]
[3100 3101]
[3102]
[3102 3103]
[3104]
[3104 3105]
[3106]
[3106 3107]
[3108]
[3108 3109]
[3110]
[3110 3111]
[3112]
[3112 3113]
[3114]
[3114 3115]
[3116]
[3116 3117]
[3118]
[3118 3119]
[3120]
[3120 3121]
[3122]
[3122 3123]
[3124]
[3124 3125]
[3126]
[3126 3127]
[3128]
[3128 3129]
[3130]
[3130 3131]
[3132]
[3132 3133]
[3134]
[3134 3135]
[3136]
[3136 3137]
[3138]
[3138 3139]
[3140]
[3140 3141]
[3142]
[3142 3143]
[3144]
[3144 3145]
[3146]
[3146 3147]
[3148]
[3148 3149]
[3150]
[3150 3151]
[3152]
[3152 3153]
[3154]
[3154 3155]
[3156]
[3156 3157]
[3158]
[3158 3159]
[3160]
[3160 3161]
[3162]
[3162 3163]
[3164]
[3164 3165]
[3166]
[3166 3167]
[3168]
[3168 3169]
[3170]
[3170 3171]


[4138 4139]
[4140]
[4140 4141]
[4142]
[4142 4143]
[4144]
[4144 4145]
[4146]
[4146 4147]
[4148]
[4148 4149]
[4150]
[4150 4151]
[4152]
[4152 4153]
[4154]
[4154 4155]
[4156]
[4156 4157]
[4158]
[4158 4159]
[4160]
[4160 4161]
[4162]
[4162 4163]
[4164]
[4164 4165]
[4166]
[4166 4167]
[4168]
[4168 4169]
[4170]
[4170 4171]
[4172]
[4172 4173]
[4174]
[4174 4175]
[4176]
[4176 4177]
[4178]
[4178 4179]
[4180]
[4180 4181]
[4182]
[4182 4183]
[4184]
[4184 4185]
[4186]
[4186 4187]
[4188]
[4188 4189]
[4190]
[4190 4191]
[4192]
[4192 4193]
[4194]
[4194 4195]
[4196]
[4196 4197]
[4198]
[4198 4199]
[4200]
[4200 4201]
[4202]
[4202 4203]
[4204]
[4204 4205]
[4206]
[4206 4207]
[4208]
[4208 4209]
[4210]
[4210 4211]
[4212]
[4212 4213]
[4214]
[4214 4215]
[4216]
[4216 4217]
[4218]
[4218 4219]
[4220]
[4220 4221]
[4222]
[4222 4223]
[4224]
[4224 4225]
[4226]
[4226 4227]
[4228]
[4228 4229]
[4230]
[4230 4231]
[4232]
[4232 4233]
[4234]
[4234 4235]
[4236]
[4236 4237]
[4238]
[4238 4239]
[4240]
[4240 4241]
[4242]
[4242 4243]


...

[34394]
[34394 34395]
[34396]
[34396 34397]
[34398]
[34398 34399]
[34400]
[34400 34401]
[34402]
[34402 34403]
[34404]
[34404 34405]
[34406]
[34406 34407]
[34408]
[34408 34409]
[34410]
[34410 34411]
[34412]
[34412 34413]
[34414]
[34414 34415]
[34416]
[34416 34417]
[34418]
[34418 34419]
[34420]
[34420 34421]
[34422]
[34422 34423]
[34424]
[34424 34425]
[34426]
[34426 34427]
[34428]
[34428 34429]
[34430]
[34430 34431]
[34432]
[34432 34433]
[34434]
[34434 34435]
[34436]
[34436 34437]
[34438]
[34438 34439]
[34440]
[34440 34441]
[34442]
[34442 34443]
[34444]
[34444 34445]
[34446]
[34446 34447]
[34448]
[34448 34449]
[34450]
[34450 34451]
[34452]
[34452 34453]
[34454]
[34454 34455]
[34456]
[34456 34457]
[34458]
[34458 34459]
[34460]
[34460 34461]
[34462]
[34462 34463]
[34464]
[34464 34465]
[34466]
[34466 34467]
[34468]
[34468 34469]
[34470]
[34470 34471]
[34472]
[34472 34473]
[34474]
[34474 34475]
[34476]
[34476 34477]
[34478]
[34478 34479]
[34480]
[34480 34481]
[34482]
[34482 34483]
[34484]
[3

[35428]
[35428 35429]
[35430]
[35430 35431]
[35432]
[35432 35433]
[35434]
[35434 35435]
[35436]
[35436 35437]
[35438]
[35438 35439]
[35440]
[35440 35441]
[35442]
[35442 35443]
[35444]
[35444 35445]
[35446]
[35446 35447]
[35448]
[35448 35449]
[35450]
[35450 35451]
[35452]
[35452 35453]
[35454]
[35454 35455]
[35456]
[35456 35457]
[35458]
[35458 35459]
[35460]
[35460 35461]
[35462]
[35462 35463]
[35464]
[35464 35465]
[35466]
[35466 35467]
[35468]
[35468 35469]
[35470]
[35470 35471]
[35472]
[35472 35473]
[35474]
[35474 35475]
[35476]
[35476 35477]
[35478]
[35478 35479]
[35480]
[35480 35481]
[35482]
[35482 35483]
[35484]
[35484 35485]
[35486]
[35486 35487]
[35488]
[35488 35489]
[35490]
[35490 35491]
[35492]
[35492 35493]
[35494]
[35494 35495]
[35496]
[35496 35497]
[35498]
[35498 35499]
[35500]
[35500 35501]
[35502]
[35502 35503]
[35504]
[35504 35505]
[35506]
[35506 35507]
[35508]
[35508 35509]
[35510]
[35510 35511]
[35512]
[35512 35513]
[35514]
[35514 35515]
[35516]
[35516 35517]
[35518]
[3

[36448]
[36448 36449]
[36450]
[36450 36451]
[36452]
[36452 36453]
[36454]
[36454 36455]
[36456]
[36456 36457]
[36458]
[36458 36459]
[36460]
[36460 36461]
[36462]
[36462 36463]
[36464]
[36464 36465]
[36466]
[36466 36467]
[36468]
[36468 36469]
[36470]
[36470 36471]
[36472]
[36472 36473]
[36474]
[36474 36475]
[36476]
[36476 36477]
[36478]
[36478 36479]
[36480]
[36480 36481]
[36482]
[36482 36483]
[36484]
[36484 36485]
[36486]
[36486 36487]
[36488]
[36488 36489]
[36490]
[36490 36491]
[36492]
[36492 36493]
[36494]
[36494 36495]
[36496]
[36496 36497]
[36498]
[36498 36499]
[36500]
[36500 36501]
[36502]
[36502 36503]
[36504]
[36504 36505]
[36506]
[36506 36507]
[36508]
[36508 36509]
[36510]
[36510 36511]
[36512]
[36512 36513]
[36514]
[36514 36515]
[36516]
[36516 36517]
[36518]
[36518 36519]
[36520]
[36520 36521]
[36522]
[36522 36523]
[36524]
[36524 36525]
[36526]
[36526 36527]
[36528]
[36528 36529]
[36530]
[36530 36531]
[36532]
[36532 36533]
[36534]
[36534 36535]
[36536]
[36536 36537]
[36538]
[3

[37464 37465]
[37466]
[37466 37467]
[37468]
[37468 37469]
[37470]
[37470 37471]
[37472]
[37472 37473]
[37474]
[37474 37475]
[37476]
[37476 37477]
[37478]
[37478 37479]
[37480]
[37480 37481]
[37482]
[37482 37483]
[37484]
[37484 37485]
[37486]
[37486 37487]
[37488]
[37488 37489]
[37490]
[37490 37491]
[37492]
[37492 37493]
[37494]
[37494 37495]
[37496]
[37496 37497]
[37498]
[37498 37499]
[37500]
[37500 37501]
[37502]
[37502 37503]
[37504]
[37504 37505]
[37506]
[37506 37507]
[37508]
[37508 37509]
[37510]
[37510 37511]
[37512]
[37512 37513]
[37514]
[37514 37515]
[37516]
[37516 37517]
[37518]
[37518 37519]
[37520]
[37520 37521]
[37522]
[37522 37523]
[37524]
[37524 37525]
[37526]
[37526 37527]
[37528]
[37528 37529]
[37530]
[37530 37531]
[37532]
[37532 37533]
[37534]
[37534 37535]
[37536]
[37536 37537]
[37538]
[37538 37539]
[37540]
[37540 37541]
[37542]
[37542 37543]
[37544]
[37544 37545]
[37546]
[37546 37547]
[37548]
[37548 37549]
[37550]
[37550 37551]
[37552]
[37552 37553]
[37554]
[37554 375

KeyboardInterrupt: 